In [2]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

[nltk_data] Downloading package punkt to /Users/mcabook/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/mcabook/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [4]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [5]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

91 documents
44 classes ['goodbye', 'greeting', 'hours', 'mopeds', 'negative', 'opentoday', 'payments', 'positive', 'problem: PLC alarm 1', 'problem: PLC alarm 10', 'problem: PLC alarm 11', 'problem: PLC alarm 12', 'problem: PLC alarm 13', 'problem: PLC alarm 14', 'problem: PLC alarm 15', 'problem: PLC alarm 16', 'problem: PLC alarm 17', 'problem: PLC alarm 18', 'problem: PLC alarm 2', 'problem: PLC alarm 3', 'problem: PLC alarm 4', 'problem: PLC alarm 5', 'problem: PLC alarm 6', 'problem: PLC alarm 7', 'problem: PLC alarm 8', 'problem: PLC alarm 9', 'problem: arm tool 1', 'problem: arm tool 2', 'problem: arm tool 3', 'problem: axis trouble 1', 'problem: axis trouble 2', 'problem: axis trouble 3', 'problem: coolant pump', 'problem: lubrication pump', 'problem: not flat', 'problem: shakes', 'problem: spindle trouble', 'problem: umbrella tool 1', 'problem: umbrella tool 2', 'problem: umbrella tool 3', 'problem: umbrella tool 4', 'problem: umbrella tool 5', 'rental', 'today']
172 unique l

In [6]:

# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


In [7]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.7))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/200
19/19 [==============================] - 0s 2ms/step - loss: 3.8502 - accuracy: 0.0110
Epoch 2/200
19/19 [==============================] - 0s 2ms/step - loss: 3.7546 - accuracy: 0.0330
Epoch 3/200
19/19 [==============================] - 0s 2ms/step - loss: 3.6965 - accuracy: 0.0440
Epoch 4/200
19/19 [==============================] - 0s 2ms/step - loss: 3.6684 - accuracy: 0.0549
Epoch 5/200
19/19 [==============================] - 0s 1ms/step - loss: 3.5837 - accuracy: 0.0879
Epoch 6/200
19/19 [==============================] - 0s 2ms/step - loss: 3.4709 - accuracy: 0.1319
Epoch 7/200
19/19 [==============================] - 0s 2ms/step - loss: 3.3709 - accuracy: 0.1648
Epoch 8/200
19/19 [==============================] - 0s 2ms/step - loss: 3.3929 - accuracy: 0.1319
Epoch 9/200
19/19 [==============================] - 0s 2ms/step - loss: 3.2305 - accuracy: 0.2418
Epoch 10/200
19/19 [==============================] - 0s 2ms/step - loss: 3.1534 - accuracy: 0.1538
Epoch 11/

19/19 [==============================] - 0s 2ms/step - loss: 0.3568 - accuracy: 0.9231
Epoch 84/200
19/19 [==============================] - 0s 2ms/step - loss: 0.2147 - accuracy: 0.9451
Epoch 85/200
19/19 [==============================] - 0s 2ms/step - loss: 0.2187 - accuracy: 0.9451
Epoch 86/200
19/19 [==============================] - 0s 2ms/step - loss: 0.1291 - accuracy: 0.9780
Epoch 87/200
19/19 [==============================] - 0s 2ms/step - loss: 0.2494 - accuracy: 0.9451
Epoch 88/200
19/19 [==============================] - 0s 2ms/step - loss: 0.3490 - accuracy: 0.8681
Epoch 89/200
19/19 [==============================] - 0s 2ms/step - loss: 0.1601 - accuracy: 0.9451
Epoch 90/200
19/19 [==============================] - 0s 2ms/step - loss: 0.2074 - accuracy: 0.9341
Epoch 91/200
19/19 [==============================] - 0s 2ms/step - loss: 0.1974 - accuracy: 0.9451
Epoch 92/200
19/19 [==============================] - 0s 2ms/step - loss: 0.2251 - accuracy: 0.9451
Epoch 93/200


19/19 [==============================] - 0s 2ms/step - loss: 0.1214 - accuracy: 0.9670
Epoch 165/200
19/19 [==============================] - 0s 3ms/step - loss: 0.1146 - accuracy: 0.9780
Epoch 166/200
19/19 [==============================] - 0s 3ms/step - loss: 0.1425 - accuracy: 0.9670
Epoch 167/200
19/19 [==============================] - 0s 2ms/step - loss: 0.1343 - accuracy: 0.9451
Epoch 168/200
19/19 [==============================] - 0s 2ms/step - loss: 0.1215 - accuracy: 0.9780
Epoch 169/200
19/19 [==============================] - 0s 2ms/step - loss: 0.1082 - accuracy: 0.9670
Epoch 170/200
19/19 [==============================] - 0s 2ms/step - loss: 0.0896 - accuracy: 0.9780
Epoch 171/200
19/19 [==============================] - 0s 2ms/step - loss: 0.2524 - accuracy: 0.8901
Epoch 172/200
19/19 [==============================] - 0s 2ms/step - loss: 0.1641 - accuracy: 0.9451
Epoch 173/200
19/19 [==============================] - 0s 2ms/step - loss: 0.1223 - accuracy: 0.9670
Epoc

In [8]:
from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [9]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

# If it's related to a particular issue, then it returns a response corresponding 
# to the most likely solution based on the 'confidence score'; if it's a general question
# then choose from the response list randomly.


def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    if tag.startswith('problem:'):
        result = sorted(prob_sol_dict[tag], key = prob_sol_dict[tag].get, reverse = True)[0]
        print(result)
        return result
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

#Matching the negative and positive feedback patterns and return "positive" or 'negative' as a string

def getFeedback(feedback, intents_json): # Returns Pos or Neg
    ints = predict_class(feedback, model)
    res = getResponse(ints, intents)
    print(ints,res)
    return res
        

def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res,ints


In [10]:
#updates the priority of each possible solution by incrementing or substracting 1 from the total count

def updatePriority(solution,prob_tag, label):
    if prob_tag == '':
        return
    if label == 'positive':
        prob_sol_dict[prob_tag][solution] += 1
        print(prob_sol_dict[prob_tag])
    if label == 'negative':
        prob_sol_dict[prob_tag][solution] -= 1
        print(prob_sol_dict[prob_tag])

In [11]:
# A new data structure to store the multiple solutions with their frequency count(confidence score)
# {problem_tag1: {solution1: count1, solution2: count2,...},
#  problem _tag2: ...}

prob_sol_dict = {}

for item in intents['intents']:
    if item['tag'].startswith('problem:'):
        count_dict = {}
        for solution in item['responses']:
            count_dict[solution] = 1
        prob_sol_dict[item['tag']] = count_dict
prob_sol_dict[''] = 'puck'
print(prob_sol_dict)

{'problem: not flat': {'Adjust the spindle bearing gaps': 1, "Adjust the gib's gap": 1, 'wiggle your toes': 1, 'Have some vodka man': 1, 'Leave the company': 1}, 'problem: shakes': {'Retighten the machine': 1, 'Choose suitable cutting feed': 1}, 'problem: umbrella tool 1': {'Check the air pressure': 1, 'Check the milling chuck pull stud model': 1, 'Check the cylindrical travel': 1, 'Check the oil input': 1}, 'problem: umbrella tool 2': {'Check the air pressure': 1, "Check the tool magazine manually if can't move, check the circuit breaker": 1, 'Check if tool magazine motor phase is right or not': 1, 'Check if the limit switch of tool loosening is in right position': 1, 'Check if the sensor switch is right or not': 1, 'Check if the tools is hit or the reference point is changing adjust 3-axis parameter according to P 2038(MITSUBISHI) or P1241(FANUC)': 1}, 'problem: umbrella tool 3': {'Check if the motor phase': 1, 'Reset the tool magazine, and re-perform the procedure': 1}, 'problem: um

In [14]:
#Creating GUI with tkinter

import smtplib
from smtplib import SMTPException
import tkinter
from tkinter import *
import tkinter.ttk as ttk
import webbrowser

current_solution = ''
current_prob_tag = ''
wrong_answer = 0

global user_email
global receiver_email
global subj
global password
global body

def send():
    global current_solution
    global current_prob_tag
    global wrong_answer
    global temp_sol_dict
    
    end_of_problem = False
    passed = False
    
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)
    

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#ffffff", font=("Verdana", 12 ))

        res, probs_dict = chatbot_response(msg)
        tag = probs_dict[0]['intent']
        
        if wrong_answer >0:
            if tag == 'positive':
                print('---Feedback running---')
                updatePriority(current_solution, current_prob_tag, tag)
                end_of_problem = True
                wrong_answer = 0

            if tag == 'negative':
                
                    
                print('---Feedback running---')
                updatePriority(current_solution, current_prob_tag, tag)
                wrong_answer += 1
                temp_sol_dict.pop(current_solution)
                if not temp_sol_dict:
                    ChatLog.insert(END, "Kwahz Bot: Please give us a feedback via email" '\n\n')
                    end_of_problem = True
                else:
                    current_solution = sorted(temp_sol_dict,key = temp_sol_dict.get, reverse = True)[0]
                    print(current_solution)
                    ChatLog.insert(END, "Bot: " + current_solution + '\n\n')
                    passed = True
                

        #Identifies problems from a message
        elif tag.startswith('problem:'):
            print(tag)
            current_solution = res
            current_prob_tag = tag
            print(current_solution)

        # Identifies feedback from a message
        elif tag == 'positive':
            print('---Feedback running---')
            updatePriority(current_solution, current_prob_tag, tag)
            end_of_problem = True
            wrong_answer = 0

        elif tag == 'negative':
            print('---Feedback running---')
            updatePriority(current_solution, current_prob_tag, tag)
            wrong_answer += 1
            
            # Now assign the alternative solution
            temp_sol_dict = prob_sol_dict[current_prob_tag].copy()
            temp_sol_dict.pop(current_solution)
            
            #Checking empty dictionary
            if not temp_sol_dict:
                ChatLog.insert(END, "Kwahz Bot: Please give us a feedback via email" '\n\n')
                end_of_problem = True
            else:
                current_solution = sorted(temp_sol_dict,key = temp_sol_dict.get, reverse = True)[0]
                print(current_solution)
                ChatLog.insert(END, "Kwahz Bot: " + current_solution + '\n\n')
                passed = True
                
        if not passed:
            ChatLog.insert(END, "Kwahz Bot: " + res + '\n\n')
        if end_of_problem:
            current_solution = ''
            current_prob_tag = ''
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)

base = Tk()
ttk.Style().theme_use('alt')
base.title("Chatbot")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

def gmail():   
    usermail = user_email.get()
    receivermail=receiver_email.get()           
    server=smtplib.SMTP('smtp.gmail.com:587')
    pass_word=password.get()
    subject=subj.get()
    #This allow you to include a subject by adding from, to and subject 
    line
    main_message=body.get('1.0', 'end-1c')
    Body="""From: Name here <usermail>
    To: <receivermail>
    Subject:%s 

    %s
    """ %(subject,main_message )

    try:
            server=smtplib.SMTP('smtp.gmail.com:587')
            server.ehlo()
            server.starttls()
            server.login(usermail, pass_word  )
            server.sendmail(usermail,receivermail, Body )

            text.insert(1.0, 'message sent')
         #error handling
    except  (smtplib.SMTPException,ConnectionRefusedError,OSError):
            text.insert(1.0, 'message not sent')

            
def openNewWindow(): 
      
    # Toplevel object which will  
    # be treated as a new window 
       
    root= Tk(className=" Gmail app " )
    root.config(bg="#242424", )
     
    
    #user mail
    user_email = Label(root, font="Arial", text="Enter your Gmail address:  ")
    user_email.pack()
    user_email.config(bg="#242424", fg="white")
    user_email = Entry(root, bd =8)
    user_email.pack(fill=X)
    
    #receiver email
    receiver_email = Label(root, font = "Arial", text="Enter the recipient's email address:")
    receiver_email.pack()
    receiver_email.config(bg="#242424", fg="white")
    receiver_email = Entry(root, bd =8)
    receiver_email.pack(fill=X)
    
    #subject line
    subj= Label(root, font = "Arial", text="Enter your subject here: ")
    subj.pack()
    subj.config(bg="#242424", fg="white")
    
    subj = Entry(root, bd =8)
    subj.pack(fill=X)

    #Body of the message
    body = Text(root, font="Arial",  relief=SUNKEN , bd=8)
    body.config(bg="#ffffff", height=15)
    body.pack(fill=BOTH, expand=True)
    
    #password widget
    password = Label(root, font="Arial", text="Enter your Gmail password:  ")
    password.pack()
    password.config(bg="#242424", fg="white")
    
    password= Entry(root, show='*', bd =8)
    password.pack(fill=X)

    #submit button
    submit_mail = Button(root, font=("Arial Narrow",12,'bold'), text="Send",  bd=8, bg="#000000", activebackground="#919191", fg='#ffffff',command=gmail)
    submit_mail.pack(fill=X)
    
def gmail():   
    usermail = user_email.get()
    receivermail=receiver_email.get()           
    server=smtplib.SMTP('smtp.gmail.com:587')
    pass_word=password.get()
    subject=subj.get()
    #This allow you to include a subject by adding from, to and subject 
    line
    main_message=body.get('1.0', 'end-1c')
    Body="""From: Name here <usermail>
    To: <receivermail>
    Subject:%s 

    %s
    """ %(subject,main_message )

    try:
            server=smtplib.SMTP('smtp.gmail.com:587')
            server.ehlo()
            server.starttls()
            server.login(usermail, pass_word  )
            server.sendmail(usermail,receivermail, Body )

            text.insert(1.0, 'message sent')
         #error handling
    except  (smtplib.SMTPException,ConnectionRefusedError,OSError):
            text.insert(1.0, 'message not sent')
            
new = 1
url = "https://www.google.com"

def openweb():
    webbrowser.open(url,new=new)

#Create Chat window
ChatLog = Text(base, bd=0, bg="#242424", fg='#ffffff', height="8", width="50", font="Calibri")

ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview)
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Arial Narrow",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#ffffff", activebackground="#919191",fg='#000000',
                    command= send)

EmailButton = Button(base, font=("Arial Narrow",12,'bold'), text="Email", width="12", height=5,
                    bd=0, bg="#ffffff", activebackground="#919191",fg='#000000',
                    command= openNewWindow)
WebsiteButton = Button(base, font=("Arial Narrow",12,'bold'), text="Visit our website", width="12", height=5,
                    bd=0, bg="#ffffff", activebackground="#919191",fg='#000000',
                    command=openweb)

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="#242424",fg='#ffffff',width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)


#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=36, height=356, width=370)
EntryBox.place(x=128, y=401, height=90, width=248)
SendButton.place(x=6, y=401, height=90)
EmailButton.place(x=320, y=10, height=20, width=50)
WebsiteButton.place(x=6, y=10, height=20, width=150)

base.mainloop()

Check the limit switch is right set in the software or not
problem: axis trouble 2
Check the limit switch is right set in the software or not
---Feedback running---
{'Check the limit switch is right set in the software or not': -2, 'Adjust or replace the ZERO block or SWITCH': -1}
Adjust or replace the ZERO block or SWITCH
---Feedback running---
{'Check the limit switch is right set in the software or not': -2, 'Adjust or replace the ZERO block or SWITCH': -2}
